In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.figure_factory as ff

%matplotlib inline

In [73]:
dft = pd.read_csv( '../technology-data.csv')
dfe = pd.read_csv( '../employer-data.csv')
dfe = dfe.fillna(2020)
dft.head()

Category     Vendor          Technology  Employer Sum  Pluralsight  \
0  APIs / Specs      OASIS                SAML             1          NaN   
1  APIs / Specs      OASIS                SPML             1          NaN   
2  APIs / Specs     Splunk  Splunk Integration             1          NaN   
3  APIs / Specs  Microsoft    SCOM Integration             1          NaN   
4  APIs / Specs     Google    Google Drive API             2          1.0   

   Yarn Hooks & Love Silly, LLC  Code Owls LLC  PokerTek, Inc  \
0                           NaN            NaN            NaN   
1                           NaN            NaN            NaN   
2                           NaN            1.0            NaN   
3                           NaN            1.0            NaN   
4                           NaN            1.0            NaN   

   LearningStation, Inc.  Convey Systems, Inc  ...  \
0                    1.0                  NaN  ...   
1                    1.0                  NaN  ...   
2                    NaN                  NaN  ...   
3                    NaN                  NaN  ...   
4                    NaN                  NaN  ...   

   Continuous Integration and Testing  Data Visualization  \
0                                 NaN                 NaN   
1                                 NaN                 NaN   
2                                 NaN                 NaN   
3                                 NaN                 NaN   
4                                 NaN                 NaN   

   Desktop Development  Embedded Systems / Hardware Development  \
0                  NaN                                      NaN   
1                  NaN                                      NaN   
2                  NaN                                      NaN   
3                  NaN                                      NaN   
4                  NaN                                      NaN   

   Natural Language Processing  Machine Learning  Programming Languages  \
0                          NaN               NaN                    NaN   
1                          NaN               NaN                    NaN   
2                          NaN               NaN                    NaN   
3                          NaN               NaN                    NaN   
4                          NaN               NaN                    NaN   

   Programming Tools  Version Control Systems  Web Development  
0                NaN                      NaN              NaN  
1                NaN                      NaN              NaN  
2                NaN                      NaN              NaN  
3                NaN                      NaN              NaN  
4                NaN                      NaN              NaN  

[5 rows x 30 columns]

In [23]:
dfe.head(10)

Employer  Start Date  End State
0                    Pluralsight        2014     2020.0
1   Yarn Hooks & Love Silly, LLC        2017     2020.0
2                  Code Owls LLC        2009     2020.0
3                  PokerTek, Inc        2005     2010.0
4          LearningStation, Inc.        2002     2005.0
5            Convey Systems, Inc        2000     2002.0
6  Breault Research Organization        1997     1999.0

In [41]:
dfd = dft[ ['Technology','Pluralsight', 'Yarn Hooks & Love Silly, LLC', 'Code Owls LLC','PokerTek, Inc', 'LearningStation, Inc.', 'Convey Systems, Inc', 'Breault Research Organization' ] ]
dfd = dfd.set_index( "Technology" )
dfd = dfd.sort_index()
dfd.head()

Pluralsight  Yarn Hooks & Love Silly, LLC  Code Owls LLC  \
Technology                                                                      
8086 Assembler               NaN                           NaN            1.0   
ASP                          NaN                           NaN            NaN   
ASP.NET                      NaN                           1.0            1.0   
Amazon Web Services          1.0                           NaN            1.0   
Arduino                      NaN                           1.0            NaN   

                     PokerTek, Inc  LearningStation, Inc.  \
Technology                                                  
8086 Assembler                 1.0                    NaN   
ASP                            NaN                    1.0   
ASP.NET                        1.0                    NaN   
Amazon Web Services            NaN                    NaN   
Arduino                        NaN                    NaN   

                     Convey Systems, Inc  Breault Research Organization  
Technology                                                               
8086 Assembler                       1.0                            NaN  
ASP                                  1.0                            NaN  
ASP.NET                              NaN                            NaN  
Amazon Web Services                  NaN                            NaN  
Arduino                              NaN                            NaN

In [42]:
plt.figure( figsize=(10,100) )
plt.imshow( dfd )
cols = ['Pluralsight', 'Yarn Hooks & Love Silly, LLC', 'Code Owls LLC','PokerTek, Inc', 'LearningStation, Inc.', 'Convey Systems, Inc', 'Breault Research Organization' ]
plt.xticks( range(len(cols)), dfd[ cols ] , rotation=90)
plt.yticks( range(len(dfd)), dfd.index)
plt.show()

In [137]:
dfd = dft[ ['Technology','Pluralsight', 'Yarn Hooks & Love Silly, LLC', 'Code Owls LLC','PokerTek, Inc', 'LearningStation, Inc.', 'Convey Systems, Inc', 'Breault Research Organization' ] ]
dfd = dfd.melt( id_vars=['Technology'], value_vars=cols, var_name='Employer', value_name='Is Used')
dfex = dfe.set_index('Employer')
dfd = dfd.join( dfex, on="Employer", how='left' )
dfd = dfd.dropna()
dfd = dfd.reset_index( drop=True)
dfd = dfd.rename(columns={'End State':'End Date'})
dfd['End Date'] = dfd[ 'End Date' ].astype('int32')
dfd.head()

Technology     Employer  Is Used  Start Date  End Date
0    Google Drive API  Pluralsight      1.0        2014      2020
1   Google Sheets API  Pluralsight      1.0        2014      2020
2     Google Docs API  Pluralsight      1.0        2014      2020
3  Google Scripts API  Pluralsight      1.0        2014      2020
4               OAuth  Pluralsight      1.0        2014      2020

In [138]:
dfd.loc[ dfd['Employer']=='Breault Research Organization', 'End Date' ] = 2000
dfd[dfd['Employer']=='Breault Research Organization'].head()


Technology                       Employer  Is Used  Start Date  End Date
219        Excel  Breault Research Organization      1.0        1997      2000
220  Windows API  Breault Research Organization      1.0        1997      2000
221    COM / ATL  Breault Research Organization      1.0        1997      2000
222   javascript  Breault Research Organization      1.0        1997      2000
223          C++  Breault Research Organization      1.0        1997      2000

In [140]:

dfg = dfd.rename(columns={'Technology':'Task', 'Start Date':'Start','End Date':'Finish'}) \
    .sort_values('Task') \
    .reset_index(drop=True)

fig = ff.create_gantt( dfg, 
                      index_col = 'Employer', 
                      show_colorbar=True, 
                      group_tasks=True, 
                      height=1500,
                      width=1000,
                     showgrid_y=True,
                     showgrid_x=True)
fig.show()